##01-INSTALAMOS E IMPORTAMOS BIBLIOTECAS


In [2]:
!pip install --upgrade requests pandas scikit-learn numpy joblib dill

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This be

In [32]:
import requests
from io import StringIO
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler
from sklearn.pipeline import Pipeline
import numpy as np
import dill
from joblib import dump

In [31]:
import sklearn
print(sklearn.__version__)

1.4.1.post1


##02-ENTRENAMOS Y EMPAQUETAMOS MODELO

In [43]:
# Función para filtrar columnas
def filtrar_columnas(df):
    columnas_a_mantener = [
        'International plan', 'Voice mail plan', 'Number vmail messages',
        'Total day minutes', 'Total day charge', 'Total eve minutes',
        'Total eve charge', 'Total night minutes', 'Total night charge',
        'Total intl minutes', 'Total intl calls', 'Churn'                  # Como se puede observar, en el prepro, NO se incluye el target
    ]
    return df[columnas_a_mantener]

# Función que transforma 'International plan' a int.
def mapear_international_plan(df):
    df_copy = df.copy()  # Crear una copia del DataFrame
    df_copy['International plan'] = df_copy['International plan'].map({'Yes': 1, 'No': 0})
    return df_copy

# Función que transforma 'Voice mail plan' a int.
def mapear_voice_mail_plan(df):
    df_copy = df.copy()  # Crear una copia del DataFrame
    df_copy['Voice mail plan'] = df_copy['Voice mail plan'].map({'Yes': 1, 'No': 0})
    return df_copy

# Función para eliminar outliers
def eliminar_outliers(df):
    columns_to_check = ['Number vmail messages', 'Total day minutes',
                             'Total day charge', 'Total eve minutes',
                             'Total eve charge', 'Total night minutes',
                             'Total night charge', 'Total intl minutes',
                             'Total intl calls']

    df_copy = df.copy()

    for column in columns_to_check:
        q1 = df_copy[column].quantile(0.25)
        q3 = df_copy[column].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr

        # Eliminar las filas con outliers solo en la columna actual
        df_copy = df_copy.drop(df_copy[(df_copy[column] < lower_bound) | (df_copy[column] > upper_bound)].index)

    return df_copy

# Función para normalizar variables
def norma_variables(df):
    columns_to_normalize = ['Number vmail messages', 'Total day minutes',
                             'Total day charge', 'Total eve minutes',
                             'Total eve charge', 'Total night minutes',
                             'Total night charge', 'Total intl minutes',
                             'Total intl calls']

    df_copy = df.copy()
    scaler = MinMaxScaler()
    df_copy[columns_to_normalize] = scaler.fit_transform(df_copy[columns_to_normalize])
    return df_copy


# Df virgen
file_id = "16ypxCIBr9wSGVEaXqWdZUfz9w4xzccwo"
download_link = f"https://drive.google.com/uc?id={file_id}"
response = requests.get(download_link)
csv_data = StringIO(response.text)
df = pd.read_csv(csv_data, encoding='utf-8')

#Aplicamos funciones
df1 = filtrar_columnas(df)
df1 = mapear_international_plan(df1)
df1 = mapear_voice_mail_plan(df1)
df1 = eliminar_outliers(df1)
df1 = norma_variables(df1)


# Definimos las características según lo ya definido en la etapa de elección de modelo (no incluido en el Prepro01).
X = df1[['International plan', 'Voice mail plan', 'Number vmail messages',
        'Total day minutes', 'Total day charge', 'Total eve minutes',
        'Total eve charge', 'Total night minutes', 'Total night charge',
        'Total intl minutes', 'Total intl calls']]
y = df1.Churn

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instanciamos el modelo RandomForestClassifier
forest_model = RandomForestClassifier()

# Fit del modelo
forest_model.fit(X_train, y_train)

RandomForestClassifier()

## 03- CREAMOS PAQUETE PREPROCESAMIENTO

In [44]:
# Función para filtrar columnas
def filtrar_columnas(df):
    columnas_a_mantener = [
        'International plan', 'Voice mail plan', 'Number vmail messages',
        'Total day minutes', 'Total day charge', 'Total eve minutes',
        'Total eve charge', 'Total night minutes', 'Total night charge',
        'Total intl minutes', 'Total intl calls'                   # Como se puede observar, en el prepro, NO se incluye el target
    ]
    return df[columnas_a_mantener]

# Función que transforma 'International plan' a int.
def mapear_international_plan(df):
    df_copy = df.copy()  # Crear una copia del DataFrame
    df_copy['International plan'] = df_copy['International plan'].map({'Yes': 1, 'No': 0})
    return df_copy

# Función que transforma 'Voice mail plan' a int.
def mapear_voice_mail_plan(df):
    df_copy = df.copy()  # Crear una copia del DataFrame
    df_copy['Voice mail plan'] = df_copy['Voice mail plan'].map({'Yes': 1, 'No': 0})
    return df_copy

# Función para eliminar outliers
def eliminar_outliers(df):
    columns_to_check = ['Number vmail messages', 'Total day minutes',
                             'Total day charge', 'Total eve minutes',
                             'Total eve charge', 'Total night minutes',
                             'Total night charge', 'Total intl minutes',
                             'Total intl calls']

    df_copy = df.copy()

    for column in columns_to_check:
        q1 = df_copy[column].quantile(0.25)
        q3 = df_copy[column].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr

        # Eliminar las filas con outliers solo en la columna actual
        df_copy = df_copy.drop(df_copy[(df_copy[column] < lower_bound) | (df_copy[column] > upper_bound)].index)

    return df_copy

# Función para normalizar variables
def norma_variables(df):
    columns_to_normalize = ['Number vmail messages', 'Total day minutes',
                             'Total day charge', 'Total eve minutes',
                             'Total eve charge', 'Total night minutes',
                             'Total night charge', 'Total intl minutes',
                             'Total intl calls']

    df_copy = df.copy()
    scaler = MinMaxScaler()
    df_copy[columns_to_normalize] = scaler.fit_transform(df_copy[columns_to_normalize])
    return df_copy



# Creación de transformadores
Best_caracter = FunctionTransformer(filtrar_columnas)
Mapeo_01 = FunctionTransformer(mapear_international_plan)
Mapeo_02 = FunctionTransformer(mapear_voice_mail_plan)
Outliers_drop = FunctionTransformer(eliminar_outliers)
Normalizacion = FunctionTransformer(norma_variables)


# Creación del pipeline de preprocesamiento
Prepro01 = Pipeline(steps=[
    ("Best_features", Best_caracter),
    ("Mapeo_01", Mapeo_01),
    ("Mapeo_02", Mapeo_02),
    ("Outliers_drop", Outliers_drop),
    ("Normalizacion", Normalizacion),
])

## 04-TESTEO DE PAQUETE DE PREPROCESAMIENTO Y MODELADO

In [45]:
#Traemos df para hacer prueba
file_id = "16ypxCIBr9wSGVEaXqWdZUfz9w4xzccwo"
download_link = f"https://drive.google.com/uc?id={file_id}"
response = requests.get(download_link)
csv_data = StringIO(response.text)
df = pd.read_csv(csv_data, encoding='utf-8')

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2666 entries, 0 to 2665
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   State                   2666 non-null   object 
 1   Account length          2666 non-null   int64  
 2   Area code               2666 non-null   int64  
 3   International plan      2666 non-null   object 
 4   Voice mail plan         2666 non-null   object 
 5   Number vmail messages   2666 non-null   int64  
 6   Total day minutes       2666 non-null   float64
 7   Total day calls         2666 non-null   int64  
 8   Total day charge        2666 non-null   float64
 9   Total eve minutes       2666 non-null   float64
 10  Total eve calls         2666 non-null   int64  
 11  Total eve charge        2666 non-null   float64
 12  Total night minutes     2666 non-null   float64
 13  Total night calls       2666 non-null   int64  
 14  Total night charge      2666 non-null   

In [47]:
# Introducimos df virgen en preprocesamiento
df1=Prepro01.transform(df)

In [48]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2501 entries, 0 to 2665
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   International plan     2501 non-null   int64  
 1   Voice mail plan        2501 non-null   int64  
 2   Number vmail messages  2501 non-null   float64
 3   Total day minutes      2501 non-null   float64
 4   Total day charge       2501 non-null   float64
 5   Total eve minutes      2501 non-null   float64
 6   Total eve charge       2501 non-null   float64
 7   Total night minutes    2501 non-null   float64
 8   Total night charge     2501 non-null   float64
 9   Total intl minutes     2501 non-null   float64
 10  Total intl calls       2501 non-null   float64
dtypes: float64(9), int64(2)
memory usage: 234.5 KB


In [49]:
# Introducimos df preprocesado a modelo
dfr=forest_model.predict(df1)
dfr

array([False, False, False, ..., False, False, False])

## DESCARGA DE PAQUETES

In [50]:
# Guardar el pipeline de preprocesamiento en un archivo .joblib
dump(Prepro01, 'Prepro01.joblib')

['Prepro01.joblib']

In [51]:
# Guardar el modelo RandomForestClassifier en un archivo .joblib
dump(forest_model, 'RandomForestClassifier.joblib')

['RandomForestClassifier.joblib']

In [ ]:
!pip freeze > requirements.txt

In [ ]:

# Función que transforma el tipo de datos de la columna "Churn"
def transf_entero(df):
    df['Churn'] = df['Churn'].astype(int)
    return df

# Función que transforma 'International plan' a int.
def mapear_international_plan(df):
    df['International plan'] = df['International plan'].map({'Yes': 1, 'No': 0})
    return df

# Función que transforma 'Voice mail plan' a int.
def mapear_voice_mail_plan(df):
    df['Voice mail plan'] = df['Voice mail plan'].map({'Yes': 1, 'No': 0})
    return df

# Función para eliminar la columna 'State'.
def eliminar_state(df):
    return df.drop('State', axis=1)

# Función para eliminar outliers
def eliminar_outliers(df):
    columns_to_check = ['Account length', 'Area code',
                        'Number vmail messages#', 'Total day minutes#', 'Total day calls',
                        'Total day charge#', 'Total eve minutes#', 'Total eve calls',
                        'Total eve charge#', 'Total night minutes#', 'Total night calls',
                        'Total night charge#', 'Total intl minutes#', 'Total intl calls#',
                        'Total intl charge', 'Customer service calls']

    df_copy = df.copy()

    for column in columns_to_check:
        q1 = df_copy[column].quantile(0.25)
        q3 = df_copy[column].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr

        # Eliminar las filas con outliers solo en la columna actual
        df_copy = df_copy.drop(df_copy[(df_copy[column] < lower_bound) | (df_copy[column] > upper_bound)].index)

    return df_copy

# Función para normalizar variables
def norma_variables(df):
    columns_to_normalize = ['Account length', 'Area code',
                             'Number vmail messages#', 'Total day minutes', 'Total day calls',
                             'Total day charge', 'Total eve minutes', 'Total eve calls',
                             'Total eve charge', 'Total night minutes', 'Total night calls',
                             'Total night charge', 'Total intl minutes', 'Total intl calls',
                             'Total intl charge', 'Customer service calls']

    df_copy = df.copy()
    scaler = MinMaxScaler()
    df_copy[columns_to_normalize] = scaler.fit_transform(df_copy[columns_to_normalize])
    return df_copy

# Función para filtrar columnas
def filtrar_columnas(df):
    columnas_a_mantener = [
        'International plan', 'Voice mail plan', 'Number vmail messages',
        'Total day minutes', 'Total day charge', 'Total eve minutes',
        'Total eve charge', 'Total night minutes', 'Total night charge',
        'Total intl minutes', 'Total intl calls'                   # Como se puede observar, en el prepro, NO se incluye el target
    ]
    return df[columnas_a_mantener]

# Creación de transformadores
Trans_entero = FunctionTransformer(transf_entero)
Mapeo_01 = FunctionTransformer(mapear_international_plan)
Mapeo_02 = FunctionTransformer(mapear_voice_mail_plan)
Drop_state = FunctionTransformer(eliminar_state)
Outliers_drop = FunctionTransformer(eliminar_outliers)
Normalizacion = FunctionTransformer(norma_variables)
Best_caracter = FunctionTransformer(filtrar_columnas)

# Creación del pipeline de preprocesamiento
Prepro01 = Pipeline(steps=[
    ("Trans_entero", Trans_entero),
    ("Mapeo_01", Mapeo_01),
    ("Mapeo_02", Mapeo_02),
    ("Drop_state", Drop_state),
    ("Outliers_drop", Outliers_drop),
    ("Normalizacion", Normalizacion),
    ("Best_features", Best_caracter)
])